In [12]:
import pandas as pd
import numpy as np

raw_file = "/home/ec2-user/SageMaker/Team-5/Fiona/TN_allinone0112.csv"
df = pd.read_csv(raw_file)

print(df.shape)

## Converting heights
df['Heightin_to_cm'] = df['Heightin'] * 2.53999863
df['Height_cm'] = df['Heightin_to_cm'] + df['Heightcm']
df.insert(df.columns.get_loc('Height_cm') + 1, 'Height_cm_units', 'cm')
df.drop(['Heightin_to_cm', 'Heightcm', 'Heightin'], axis=1, inplace=True)

df['Weightlbs_to_kg'] = df['Weightlbs'] * 0.45359237
df['Weight_kg'] = df['Weightlbs_to_kg'] + df['Weightkg']
df.insert(df.columns.get_loc('Weight_kg') + 1, 'Weight_kg_units', 'kg')
df.drop(['Weightlbs_to_kg', 'Weightkg', 'Weightlbs'], axis=1, inplace=True)

## Converting glucose units and organizing columns
for column in df.columns:
    if 'mmol' in column.lower() or 'mgdl' in column.lower():
        new_column_name = column.replace('mmol', '_unit').replace('mgdl', '').replace('Mmoll', '_unit').replace('MgDL', '').replace('mmoll', '_unit')
        df.rename(columns={column: new_column_name}, inplace=True)
        
columns_to_process = ['AvgRecordedFastingGlucoses', 'AvgAllRecordedGlucoses', 'LowestGlucose', 'HighestGlucose']
df[columns_to_process] = df[columns_to_process].apply(lambda x: x.where(x >= 45, x * 18.018))

columns_to_convert = ['AvgRecordedFastingGlucoses_unit', 'AvgAllRecordedGlucoses_unit', 'LowestGlucose_unitl', 'HighestGlucose_unit']
df[columns_to_convert] = df[columns_to_convert].replace('mmol/L', 'mg/dl')

Hemoglobin_process = ['HemoglobinResults']
df[Hemoglobin_process] = df[Hemoglobin_process].apply(lambda x: x.where(x <= 30, x / 10))

Hematocrit_process = ['HematocritResults']
df[Hematocrit_process] = df[Hematocrit_process].apply(lambda x: x.where(x >= 1, x * 100))

column_to_move1 = 'AvgRecordedFastingGlucoses_unit'
target_position1 = 'AvgRecordedFastingGlucoses'
current_positions = df.columns.tolist()
current_positions.remove(column_to_move1)
target_index = current_positions.index(target_position1)
new_positions = current_positions[:target_index + 1] + [column_to_move1] + current_positions[target_index + 1:]
df = df[new_positions]

column_to_move2 = 'AvgAllRecordedGlucoses_unit'
target_position2 = 'AvgAllRecordedGlucoses'
current_positions = df.columns.tolist()
current_positions.remove(column_to_move2)
target_index = current_positions.index(target_position2)
new_positions = current_positions[:target_index + 1] + [column_to_move2] + current_positions[target_index + 1:]
df = df[new_positions]

df.replace('mg/dl', 'mg/dL', inplace=True, regex=True)

df.rename(columns={'LowestGlucose_unitl': 'LowestGlucose_unit'}, inplace=True)

print(df.shape)

/tmp/ipykernel_10894/1556228860.py:5: DtypeWarning: Columns (3,4,5,6,10,15,17,21,22,24,25,27,28,30,31,33,35,36,38,40,41,44,45,48,49,52,53,56,57,60,61,62,63,64,65,66,67,68,69,70,71,72,73,75,78,81,82,83,85,86,94,107,108,109,110,111,112,113,114,115,117,119,122,126,128,129,130,132,133,137,138,142,143,147,148,152,153,157,158,162,163,167,168,169,170,171,172,173,175,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,253,256,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,300,302,303,304,305,307,310,313,317,319,322,324,325,328,330,332,334,335,336,338,343,348,349,351,352,353,354,355,356,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,38

(259168, 661)
(259168, 661)


In [13]:
existing_column_index = df.columns.get_loc('HemoglobinResults')
df.insert(existing_column_index + 1, 'HemoglobinResults_Units', "NaN")
df.loc[df['HemoglobinResults'].notna(), 'HemoglobinResults_Units'] = "mg/dL"

existing_column_index = df.columns.get_loc('HbA1c')
df.insert(existing_column_index + 1, 'HbA1c_Units', "NaN")
df.loc[df['HbA1c'].notna(), 'HbA1c_Units'] = "Hb%"

existing_column_index = df.columns.get_loc('MCHresults')
df.insert(existing_column_index + 1, 'MCHresults_Units', "NaN")
df.loc[df['MCHresults'].notna(), 'MCHresults_Units'] = "pg"

existing_column_index = df.columns.get_loc('MCHCresults')
df.insert(existing_column_index + 1, 'MCHCresults_Units', "NaN")
df.loc[df['MCHCresults'].notna(), 'MCHCresults_Units'] = "g/dL"

existing_column_index = df.columns.get_loc('EosinophilsResults')
df.insert(existing_column_index + 1, 'EosinophilsResults_Units', "NaN")
df.loc[df['EosinophilsResults'].notna(), 'EosinophilsResults_Units'] = "% of WBC"

existing_column_index = df.columns.get_loc('BasophilsResults')
df.insert(existing_column_index + 1, 'BasophilsResults_Units', "NaN")
df.loc[df['BasophilsResults'].notna(), 'BasophilsResults_Units'] = "% of WBC"

existing_column_index = df.columns.get_loc('LymphocytesResults')
df.insert(existing_column_index + 1, 'LymphocytesResults_Units', "NaN")
df.loc[df['LymphocytesResults'].notna(), 'LymphocytesResults_Units'] = "% of WBC"

existing_column_index = df.columns.get_loc('HematocritResults')
df.insert(existing_column_index + 1, 'HematocritResults_Units', "NaN")
df.loc[df['HematocritResults'].notna(), 'HematocritResults_Units'] = "% of blood"

existing_column_index = df.columns.get_loc('MCVresults')
df.insert(existing_column_index + 1, 'MCVresults_Units', "NaN")
df.loc[df['MCVresults'].notna(), 'MCVresults_Units'] = "fL"

existing_column_index = df.columns.get_loc('MonocytesResults')
df.insert(existing_column_index + 1, 'MonocytesResults_Units', "NaN")
df.loc[df['MonocytesResults'].notna(), 'MonocytesResults_Units'] = "% of WBC"

existing_column_index = df.columns.get_loc('NeutrophilsResults')
df.insert(existing_column_index + 1, 'NeutrophilsResults_Units', "NaN")
df.loc[df['NeutrophilsResults'].notna(), 'NeutrophilsResults_Units'] = "% of WBC"

existing_column_index = df.columns.get_loc('WBCresults')
df.insert(existing_column_index + 1, 'WBCresults_Units', "NaN")
df.loc[df['WBCresults'].notna(), 'WBCresults_Units'] = "x10^9 cells/L"

existing_column_index = df.columns.get_loc('Rbcresults')
df.insert(existing_column_index + 1, 'Rbcresults_Units', "NaN")
df.loc[df['Rbcresults'].notna(), 'Rbcresults_Units'] = "x10^12 cells/L"

existing_column_index = df.columns.get_loc('EosinophilsResultsOR')
df.insert(existing_column_index + 1, 'EosinophilsResultsOR_Units', "NaN")
df.loc[df['EosinophilsResultsOR'].notna(), 'EosinophilsResultsOR_Units'] = "% of WBC"

existing_column_index = df.columns.get_loc('LymphocytesResultsOR')
df.insert(existing_column_index + 1, 'LymphocytesResultsOR_Units', "NaN")
df.loc[df['LymphocytesResultsOR'].notna(), 'LymphocytesResultsOR_Units'] = "% of WBC"

existing_column_index = df.columns.get_loc('BasophilsResultsOR')
df.insert(existing_column_index + 1, 'BasophilsResultsOR_Units', "NaN")
df.loc[df['BasophilsResultsOR'].notna(), 'BasophilsResultsOR_Units'] = "% of WBC"

existing_column_index = df.columns.get_loc('MonocytesResultsOR')
df.insert(existing_column_index + 1, 'MonocytesResultsOR_Units', "NaN")
df.loc[df['MonocytesResultsOR'].notna(), 'MonocytesResultsOR_Units'] = "% of WBC"

existing_column_index = df.columns.get_loc('NeutrophilsResultsOR')
df.insert(existing_column_index + 1, 'NeutrophilsResultsOR_Units', "NaN")
df.loc[df['NeutrophilsResultsOR'].notna(), 'NeutrophilsResultsOR_Units'] = "% of WBC"

In [14]:
columns_to_filter = ['Rbcresults', 'HemoglobinResults',
                  'HematocritResults', 'MCVresults',
                  'PlateletCountResultsNormal', 'MCHresults',
                  'MCHCresults', 'WBCresults',
                  'NeutrophilsResultsOR', 'LymphocytesResultsOR',
                  'MonocytesResultsOR', 'EosinophilsResultsOR',
                  'BasophilsResultsOR']
filter_columns = df.columns[df.columns.str.contains('ResultsNormal', case=False)].tolist()
element_to_remove = 'PlateletCountResultsNormal'
filter_columns.remove(element_to_remove)

print(columns_to_filter)
print(filter_columns)

for col, filter_col in zip(columns_to_filter, filter_columns):
    filter_mask = df[filter_col] == 'Yes'
    lower_bound = df[col].quantile(0.0025)
    upper_bound = df[col].quantile(0.9975)
     # Apply np.where only to the relevant subset
    df.loc[~filter_mask, col] = np.where((df.loc[~filter_mask, col] < lower_bound) | 
                                         (df.loc[~filter_mask, col] > upper_bound), 
                                         np.nan, 
                                         df.loc[~filter_mask, col])
print(df)

['Rbcresults', 'HemoglobinResults', 'HematocritResults', 'MCVresults', 'PlateletCountResultsNormal', 'MCHresults', 'MCHCresults', 'WBCresults', 'NeutrophilsResultsOR', 'LymphocytesResultsOR', 'MonocytesResultsOR', 'EosinophilsResultsOR', 'BasophilsResultsOR']
['RBCResultsNormal', 'HemoglobinResultsNormal', 'HematocritResultsNormal', 'MCVresultsNormal', 'PlateletCountResultsNormal2', 'MCHresultsNormal', 'MCHCresultsNormal', 'WBCresultsNormal', 'NeutrophilsResultsNormal', 'LymphocytesResultsNormal', 'MonocytesResultsNormal', 'EosinophilsResultsNormal', 'BasophilsResultsNormal']
        MaskID      Visit    Visit_Dt _15Aug2011ConsentForResidualSa  \
0       200000  Screening  06/29/2007                            NaN   
1       200003  Screening  06/29/2015                            Yes   
2       200004  Screening  08/11/2015                            Yes   
3       200007  Screening  11/21/2016                            Yes   
4       200010  Screening  10/21/2016                    